# Addition

In [1]:
import torch
import ttnn

## Open the device

Use `ttnn.open` to get a handle to the device

In [2]:
device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening user mode device driver
2023-12-05 02:52:17.919 | INFO     | SiliconDriver   - Detected 1 PCI device
2023-12-05 02:52:17.929 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-12-05 02:52:17.929 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-12-05 02:52:17.929 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-12-05 02:52:17.930 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is no

## Configuration

In [3]:
h = 31
w = 22

## Initialize tensors a and b with random values using torch

To create a tensor that can be used by a `ttnn` operation:
1. Create a tensor using torch
2. Use `ttnn.from_torch` to convert the tensor from `torch.Tensor` to `ttnn.Tensor`
3. Copy the tensor onto to the device using `to_device`

In [4]:
torch.manual_seed(0)

torch_a = torch.rand((h, w), dtype=torch.bfloat16)
torch_b = torch.rand((h, w), dtype=torch.bfloat16)

a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

## Add tensor a and b

`ttnn` supports operator overloading, therefore operator `+` can be used instead of `torch.add`

In [5]:
output = a + b

                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in     0.478390974 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in     0.000452288 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinary                        finished in     0.412887095 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::UntilizeWithUnpadding                finished in     0.465376945 seconds


## Inspect the result of the add

In [6]:
print(f"shape: {output.shape}")
print(f"dtype: {output.dtype}")
print(f"layout: {output.layout}")
print(f"first row: {output[:1]}")

shape: [31, 22]
dtype: DataType.BFLOAT16
layout: Layout.ROW_MAJOR
first row: Tensor([ [0.851562, 0.183594, 0.863281, 1.45312, 0.410156, 1.32812, 1.05469, 1.35156, 0.917969, 1.30469, 0.421875, 1.32031, 0.464844, 0.441406, 1.07812, 0.96875, 0.792969, 1.32031, 0.605469, 1.42969, 1.13281, 0.375]], dtype=bfloat16 )



## Close the device

Close the handle the device. This is a very important step as the device can hang if not closed properly

In [7]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
